<a href="https://colab.research.google.com/github/lasyaEd/dsba6122-summer2024/blob/main/CreditRiskAssistantAgenticAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas openai

In [2]:
import openai
from google.colab import userdata

# Load securely
openai.api_key = userdata.get('OPENAI_API_KEY')


In [4]:
import time
import pandas as pd
import openai
import os

# — Stub/data‐fetching tools —
def fetch_basic_profile(customer_id):
    # In reality, call a DB/API. Here, we simulate with a dict.
    profiles = {
        1: {"age": 28, "income": 60000, "credit_score": 720, "debt": 5000},
        2: {"age": 34, "income": 45000, "credit_score": 680},  # debt missing
        3: {"age": 45, "income": 100000, "credit_score": 800, "debt": 10000},
    }
    return profiles.get(customer_id, {})

def fetch_debt_info(customer_id):
    # Simulate a secondary API call
    return {"debt": 15000}

# — Core scoring function (as before) —
def credit_risk_score(data, bias=0):
    score = (
        bias
      + 0.4 * data["credit_score"]
      + 0.3 * (data["income"] / 1000)
      - 0.2 * (data["debt"]   / 1000)
    )
    if score >= 400:
        return "Approve"
    elif score >= 300:
        return "Request Docs"
    else:
        return "Reject"

# — LLM explanation function —
def llm_explain(data, decision):
    prompt = f"""
    Given customer data {data}, the system decided to {decision}.
    Explain in two sentences why.
    """
    resp = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role":"user","content":prompt}],
        temperature=0.2
    )
    return resp.choices[0].message.content.strip()

# — The Agentic Loop —
def credit_agent(customer_id, max_steps=3):
    data = fetch_basic_profile(customer_id)
    steps = 0

    # 1) Loop until all required fields present or we hit max_steps
    while any(k not in data for k in ["age","income","credit_score","debt"]) and steps < max_steps:
        missing = [k for k in ("age","income","credit_score","debt") if k not in data]
        print(f"[Agent] Missing fields: {missing}")

        # 2) Choose which tool to call based on what’s missing
        if "debt" in missing:
            print("[Agent] Fetching debt info...")
            data.update(fetch_debt_info(customer_id))
        else:
            print("[Agent] No specialized tool for missing fields—proceeding with what we have.")
            break

        steps += 1
        time.sleep(0.5)  # simulate latency

    # 3) Score & decide
    decision = credit_risk_score(data)
    print(f"[Agent] Final decision: {decision}")

    # 4) Explain via LLM
    explanation = llm_explain(data, decision)
    return {"id": customer_id, "data": data, "decision": decision, "explanation": explanation}

# — Example Run —
if __name__ == "__main__":
    result = credit_agent(2)
    print("\nResult:", result)


[Agent] Missing fields: ['debt']
[Agent] Fetching debt info...
[Agent] Final decision: Reject

Result: {'id': 2, 'data': {'age': 34, 'income': 45000, 'credit_score': 680, 'debt': 15000}, 'decision': 'Reject', 'explanation': "The customer's income might not be sufficient to comfortably manage their existing debt and potential additional loan payments. Furthermore, their credit score is not high enough, indicating they may have had difficulties managing credit in the past."}
